# Part V: Natural language processing


![](img/nlp.jpeg)

In this section we'll learn:
- Preprocessing natural language data
- Document classification with Naive Bayes
- Topic modelling with Latent Dirichlet Allocation
- Document similarities

In [1]:
import os
import pandas

bio = pandas.read_parquet(os.path.join('data', 'bio.parquet'))

Natural language processing deals with documents. A usual representation of documents is the list of words it contains. A first step will be to transform the original paragraphs, to the list of words it contains.

We will loose some information in the process, but it dealing with documents directly is rarely feasible for machines.

**Exercise:** Split the documents in the column `text` into words (create a new column `words`). Take a look at one of the examples. Is there anything you do not like in it?

We will try to be smarter, and divide the words in a "smarter" way. The process of dividing the documents into words is named `tokenize`, and the `nltk` library is commonly used to for it.

`nltk` requires often data sources, for example to `tokenize`. To obtain them, the usal way is to simply use:

```
import nltk

nltk.download('punkt')
```

To avoid Internet problems, the data is already provided, and it will simply be copied to the path where `nltk` would download it.

**Exercise:** Convert the documents to lower case first, and then tokenize them using `nltk.word_tokenize` (remember that to apply an arbitrary function to all the elements of a `Series` we can use the method `.apply()`.

In [8]:
import os
import shutil

shutil.copytree(os.path.join('data', 'nltk_data'), os.path.join(os.path.expanduser('~'), 'nltk_data'))

'/home/mgarcia/nltk_data'

It is common to use other techniques to preprocess the data. For example:
- Removing common words such as `the`, `of`... (stop words)
- Merging words with common lexems, such as `organizing`, `organizer`, `organization`... (stemming and lemmatization)
- Combining words, so some context is not lost, like in `Paris Hilton` or `Paris France` (n-grams)
- Adding grammatical information to words, like in `go (noun)` (the board game Go) and `go (verb)` (POS tagging)

And others. When these can be useful depends on the problem. We will not apply any to our data, mainly for time constraints and to keep things simple.

We have seen before, that we can use `pandas.str.get_dummies()` to get one column per word, and to have a representation that we can use in `scikit-learn`.

With that, we could build a classifier that based on the biogrpahy words, predicts whether the person is a recent actress/actor or not... The most common model in `scikit-learn` when using words as features is `sklearn.naive_bayes.GaussianNB`.

With the materials already covered, it should be easy to implement such a model. That will be left as a home exercise.

Next, we will use some techniques implemented in `gensim`. To work with the library, we need to convert our data to the `gensim` format, as this is optimized to work with NLP data.

A dictionary is just a mapping of words to integers, so we do not need to save every occurrence of a word as a string (which is not efficient).

**Exercise:** Create a `gensim` dictionary by creating an instance of `gensim.corpora.Dictionary` with a list of documents as a parameter. A document is a list of words in this case.

Next, we will create a corpus using the dictionary we created. This is done by simply calling the `.doc2bow()` method of our dictionary with the document (again, the list of words) as a parameter.

`BOW` stands for bag of words. Meaning that in this step we will loose the order of the words, and we will just consider how many times each of them appears in every document. That is one of the most common techniques to deal with documents.

**Exercise:** Create a corpus by converting every element of the list of sentences to a `gensim` bag of words representation.

To get an idea of how the corpus is represented, you can execute `gensim.matutils.corpus2dense(corpus, num_terms=len(dictionary))`

One common technique to use is TF/IDF. The general idea is that a word that appears in all the documents, is not as relevant as a rare word that appears in a specific document. TF/IDF weights the importance of the words in a way that it generally helps find better patterns.

**Exercises:**
- Create a TF/IDF model for our corpus. This can be done by calling the constructor of `gensim.models.TfidfModel` with the corpus as a paramter.
- Transform our corpus based on the TF/IDF model. In `gensim`, the syntax for it is just `new_corpus = model[original_corpus]`

One of the most interesting models in natural language processing is the *Latent Direchlet Allocation (LDA)*

The idea is that the there are different topics, and every document is generated as a combination of them. For example, a movie storyline could be "generated" 20% from the topic "crime" and 80% from the topic "love".

Then, each word has also a probability of being generated from each topic. For example the word "steal", "diamond", "jail"... would have a high probability of being generated form the topic "crime", but low probability of being genreated by the topic "love.

**Exercises:**
- Create an LDA model for our corpus. This is done with the same syntax as the TF/IDF model, but in this case the constructor expects a parameter `id2word` which is the dictionary (mapping of words to positions) we created before. And also the `num_topics`, which can be any number greater than 1 (but it can be a good idea to start with a small number like 2).
- Transform our corpus to its LDA representation.

The LDA model instance has a method `.print_topics()` which show the words that the topic generates with higher probability (with the parameter `num_words` we can choose how many words to display).

It also contains the method `.show_topic()` that receives the number of topic to obtain, and a parameter `topn` to define how many words to obtain. This is returned as a list of tuples, which can be parsed easier than the output of `.print_topics()`.

**Exercise:** Get the top N words for each topic, and remove from them to ones that appear in all topics. Does this help clarify what are the topics about?

Besides the words associated to topics, we have that every document is associated to each topic with a probability. This is what we wanted, to somehow group the documents in topics. But it can also be useful to better understand the topics.

We can for example check which is the document that is more strongly associated with each topic. In this case, which is the person that is most representative of the topics created from the biographies.

**Exercise:** Transform the LDA topic percentages of the corpus, to a `DataFrame`, and set the index to the index of original corpus `DataFrame`, so we can identify every row.

If we join our topics `DataFrame` with our dataset of people, we can see who is who in the data.

**Exercise:** Load the `people.pkl` dataset, and join it with the topics `DataFrame` we just constructed. Then check which are the bios / people who belong more to each topic. Does the data make sense? Can you understand better what each topic is representing?

In [10]:
import os
import pandas

people = pandas.read_pickle(os.path.join('data', 'people.pkl'))